In [1]:
# Install required packages.
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

     |████████████████████████████████| 7.9 MB 5.5 MB/s 
     |████████████████████████████████| 3.5 MB 6.7 MB/s 
     |████████████████████████████████| 407 kB 7.2 MB/s 
     |████████████████████████████████| 41 kB 774 kB/s 


# Node Classification with Graph Neural Networks

This tutorial will teach you how to apply **Graph Neural Networks (GNNs) to the task of node classification**.
Here, we are given the ground-truth labels of only a small subset of nodes, and want to infer the labels for all the remaining nodes (*transductive learning*).

To demonstrate, we make use of the `Pubmed` dataset, which is a **citation network** where nodes represent documents.
Each node is described by a 500-dimensional bag-of-words feature vector.
Two documents are connected if there exists a citation link between them.
The task is to infer the category of each document (3 in total).

In [1]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='data/Planetoid', name='Pubmed', transform=NormalizeFeatures())

print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Processing...



Dataset: Pubmed():
Number of graphs: 1
Number of features: 500
Number of classes: 3

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])
Number of nodes: 19717
Number of edges: 88648
Average node degree: 4.50
Number of training nodes: 60
Training node label rate: 0.00
Has isolated nodes: False
Has self-loops: False
Is undirected: True


Done!


## Training a Graph Convolution Network (GCN)




In [2]:
from torch_geometric.nn import GCNConv
import torch
from torch.nn import Linear
import torch.nn.functional as F

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GCN(hidden_channels=16)
print(model)

GCN(
  (conv1): GCNConv(500, 16)
  (conv2): GCNConv(16, 3)
)


Let's train our model now!

In [3]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x, data.edge_index)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test():
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc


for epoch in range(1, 101):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Loss: 1.0991
Epoch: 002, Loss: 1.0922
Epoch: 003, Loss: 1.0865
Epoch: 004, Loss: 1.0775
Epoch: 005, Loss: 1.0687
Epoch: 006, Loss: 1.0552
Epoch: 007, Loss: 1.0541
Epoch: 008, Loss: 1.0433
Epoch: 009, Loss: 1.0276
Epoch: 010, Loss: 1.0165
Epoch: 011, Loss: 1.0102
Epoch: 012, Loss: 0.9949
Epoch: 013, Loss: 0.9852
Epoch: 014, Loss: 0.9748
Epoch: 015, Loss: 0.9533
Epoch: 016, Loss: 0.9447
Epoch: 017, Loss: 0.9204
Epoch: 018, Loss: 0.9012
Epoch: 019, Loss: 0.9108
Epoch: 020, Loss: 0.9059
Epoch: 021, Loss: 0.8768
Epoch: 022, Loss: 0.8435
Epoch: 023, Loss: 0.8474
Epoch: 024, Loss: 0.8093
Epoch: 025, Loss: 0.7907
Epoch: 026, Loss: 0.8050
Epoch: 027, Loss: 0.8111
Epoch: 028, Loss: 0.7745
Epoch: 029, Loss: 0.7735
Epoch: 030, Loss: 0.7507
Epoch: 031, Loss: 0.7315
Epoch: 032, Loss: 0.7332
Epoch: 033, Loss: 0.7078
Epoch: 034, Loss: 0.6625
Epoch: 035, Loss: 0.7063
Epoch: 036, Loss: 0.6926
Epoch: 037, Loss: 0.6372
Epoch: 038, Loss: 0.6071
Epoch: 039, Loss: 0.6383
Epoch: 040, Loss: 0.6324


After training the model, we can check its test accuracy:

In [4]:
test_acc = test()
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.7850


## Training a Graph Attention Network (GAT)




In [5]:
from torch_geometric.nn import GATConv

class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GATConv(dataset.num_features, hidden_channels, heads=heads, dropout=0.6)
        self.conv2 = GATConv(heads*hidden_channels, dataset.num_classes,concat=False, heads=heads,dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GAT(hidden_channels=8, heads=8)
print(model)

GAT(
  (conv1): GATConv(500, 8, heads=8)
  (conv2): GATConv(64, 3, heads=8)
)


In [6]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GAT(hidden_channels=8,heads=8)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x,data.edge_index)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test(mask):
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      correct = pred[mask] == data.y[mask]  # Check against ground-truth labels.
      acc = int(correct.sum()) / int(mask.sum())  # Derive ratio of correct predictions.
      return acc


for epoch in range(1, 251):
    loss = train()
    val_acc = test(data.val_mask)
    test_acc = test(data.test_mask)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Loss: 1.1005, Val: 0.3620, Test: 0.3550
Epoch: 002, Loss: 1.0936, Val: 0.5480, Test: 0.5440
Epoch: 003, Loss: 1.0957, Val: 0.7480, Test: 0.7230
Epoch: 004, Loss: 1.0913, Val: 0.7600, Test: 0.7330
Epoch: 005, Loss: 1.0885, Val: 0.7500, Test: 0.7290
Epoch: 006, Loss: 1.0882, Val: 0.7560, Test: 0.7330
Epoch: 007, Loss: 1.0861, Val: 0.7640, Test: 0.7300
Epoch: 008, Loss: 1.0813, Val: 0.7620, Test: 0.7080
Epoch: 009, Loss: 1.0769, Val: 0.7000, Test: 0.6650
Epoch: 010, Loss: 1.0781, Val: 0.6860, Test: 0.6610
Epoch: 011, Loss: 1.0732, Val: 0.7080, Test: 0.6850
Epoch: 012, Loss: 1.0697, Val: 0.7300, Test: 0.7040
Epoch: 013, Loss: 1.0651, Val: 0.7540, Test: 0.7180
Epoch: 014, Loss: 1.0611, Val: 0.7540, Test: 0.7180
Epoch: 015, Loss: 1.0550, Val: 0.7440, Test: 0.7160
Epoch: 016, Loss: 1.0493, Val: 0.7380, Test: 0.7160
Epoch: 017, Loss: 1.0482, Val: 0.7420, Test: 0.7160
Epoch: 018, Loss: 1.0469, Val: 0.7380, Test: 0.7110
Epoch: 019, Loss: 1.0390, Val: 0.7340, Test: 0.7040
Epoch: 020, 

Epoch: 159, Loss: 0.2216, Val: 0.7980, Test: 0.7850
Epoch: 160, Loss: 0.2220, Val: 0.7980, Test: 0.7830
Epoch: 161, Loss: 0.2258, Val: 0.8000, Test: 0.7870
Epoch: 162, Loss: 0.2022, Val: 0.7860, Test: 0.7850
Epoch: 163, Loss: 0.2175, Val: 0.7820, Test: 0.7800
Epoch: 164, Loss: 0.2061, Val: 0.7840, Test: 0.7810
Epoch: 165, Loss: 0.2165, Val: 0.7820, Test: 0.7820
Epoch: 166, Loss: 0.2564, Val: 0.7820, Test: 0.7840
Epoch: 167, Loss: 0.2077, Val: 0.7900, Test: 0.7870
Epoch: 168, Loss: 0.1866, Val: 0.7860, Test: 0.7850
Epoch: 169, Loss: 0.2391, Val: 0.7920, Test: 0.7810
Epoch: 170, Loss: 0.2128, Val: 0.7940, Test: 0.7790
Epoch: 171, Loss: 0.2104, Val: 0.7920, Test: 0.7810
Epoch: 172, Loss: 0.2003, Val: 0.7900, Test: 0.7800
Epoch: 173, Loss: 0.2262, Val: 0.7920, Test: 0.7810
Epoch: 174, Loss: 0.2388, Val: 0.7960, Test: 0.7810
Epoch: 175, Loss: 0.2218, Val: 0.7880, Test: 0.7820
Epoch: 176, Loss: 0.2567, Val: 0.7920, Test: 0.7850
Epoch: 177, Loss: 0.2714, Val: 0.7960, Test: 0.7860
Epoch: 178, 

In [7]:
test_acc = test(data.test_mask)
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.7820
